In [43]:
import json
import numpy as np
import pandas as pd
import time
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
import gc

In [5]:
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/lab/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install optuna

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/lab/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cpu.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cpu.html
You should consider upgrading via the '/lab/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch

In [9]:
def from_jsonl_to_df(path):
    t_start = time.time()
    with open(path, 'r') as json_file:
        json_list = list(json_file)
    json_list = [json.loads(json_str) for json_str in json_list]
    df = pd.DataFrame.from_dict(json_list, orient='columns')
    
    print("DataFrame of {}".format(len(df))+" rows loaded in {:.2f} sec".format(time.time()-t_start))
    return df

def from_csv_to_df(path):
    t_start=time.time()
    if "followers" in path:
        df=pd.read_csv(path, names=["follower", "person followed", "timestamp"], header=None)
    else:
        df=pd.read_csv(path)
    print("DataFrame of {}".format(len(df))+" rows loaded in {:.2f} sec".format(time.time()-t_start))
    return df

In [10]:
df_keaks = from_jsonl_to_df("data/full/keaks.jsonl")

DataFrame of 274472 rows loaded in 5.66 sec


In [11]:
df_beats =from_jsonl_to_df("data/full/beats.jsonl")

DataFrame of 59761 rows loaded in 1.46 sec


In [12]:
df_users =from_jsonl_to_df("data/full/users.jsonl")

DataFrame of 1162572 rows loaded in 38.74 sec


In [13]:
df_followers = from_csv_to_df("data/full/followers.csv")

DataFrame of 10506442 rows loaded in 14.91 sec


In [14]:
df_audiences = from_csv_to_df("data/full/audiences.csv")

DataFrame of 28952275 rows loaded in 42.03 sec


In [15]:
df_keaks.head(2)

,keakId,createdAt,likeCount,commentCount,viewCount,averageViewProgress,duration,hashtags,contentType,hasSmallThumbnail,link,beatId,beatGenres
0,17301813623064450175,2019-10-10T21:20:59.2970322Z,4,2,42,7.80,107.0,"[91, 1, Rap, Pen, freestyle2019, Trap2K19]",freestyle,True,https://www.keakr.com/fr/keak/mon-mec-rap-1,17301813622843862243,"[{'id': '9920897543', 'name': 'Trap'}]"
1,6202649352,2018-04-02T15:06:34.9851924Z,2,2,53,0.00,68.0,[],freestyle,True,https://www.keakr.com/fr/keak/petit-salaire,None,None


In [16]:
(df_keaks.beatId.isna()*1).sum()/df_keaks.shape[0]*100

46.74538750765105

In [17]:
df_audiences.head(2)

,userId,contentId,timestamp,progress,liked,commented,shared
0,users/6512051967,keaks/17301813623657913783,2020-01-01T00:00:00.4081666,0,True,True,True
1,users/17301813623701852659,keaks/17301813623464753700,2020-01-01T00:00:00.8245932,0,False,False,False


In [18]:
#df_audiences.loc[:,['userId','contentId']] = df_audiences.loc[:,['userId','contentId']].apply(lambda x : x[5:],axis=1)

In [19]:
df_audiences.head()

,userId,contentId,timestamp,progress,liked,commented,shared
0,users/6512051967,keaks/17301813623657913783,2020-01-01T00:00:00.4081666,0,True,True,True
1,users/17301813623701852659,keaks/17301813623464753700,2020-01-01T00:00:00.8245932,0,False,False,False
2,users/6512051967,keaks/17301813623650830605,2020-01-01T00:00:05.8421207,0,True,True,True
3,users/17301813623705778269,keaks/6102616636,2020-01-01T00:00:06.9566707,0,False,False,False
4,users/17301813623705771853,keaks/17301813622897398300,2020-01-01T00:00:08.3291103,0,False,False,False


In [20]:
df_followers.head(2)

,follower,person followed,timestamp
0,users/17301813624860662494,users/17301813625195354624,2020-06-13T16:53:19.7077441Z
1,users/11928492392,users/12680019689,2019-04-11T22:23:51.9010202Z


In [21]:
df_beats.head(2)

,beatId,genres,moods,nbKeaks,nbLikes,beatmakerId,duration,bpm,createdAt,updatedAt,link,licenceType
0,17301813628927249101,"[{'id': '9920897543', 'name': 'Trap'}]","[{'id': '17301813622132424287', 'name': 'Dark'...",7,4,17301813625134492069,121.0,102.0,2021-09-20T18:16:47.1420645Z,2021-09-20T18:17:00.2684235Z,https://keakr.com/fr/beat/turquoiz,[free]
1,17301813627622625982,"[{'id': '9920897543', 'name': 'Trap'}]",[],1,5,17301813627622569830,163.0,125.0,2021-03-31T15:20:10.726762Z,2021-03-31T15:20:26.5760601Z,https://keakr.com/fr/beat/moula-i,[free]


In [22]:
df_users.head(2)

,userId,createdAt,lastConnection,usedGenres,listenedGenres,battleCreatedCount,battleLostCount,battleRespondedCount,battleWonCount,friendCount,...,mutualFollowCount,overallBeatUsage,PlaylistCount,prizeMoneyParticipationCount,prizeMoneyWinner,sessionCount,shareCount,viewCount,isBeatmaker,isSinger
0,12354401148,2019-03-02T10:23:33.0903001Z,2019-03-02T10:23:44.3283191Z,[],[],0,0,0,0,0,...,0.0,0.0,NaN,0,False,2.0,NaN,0,False,False
1,12354411487,2019-03-02T10:26:23.9016307Z,2020-09-24T06:16:46.8631984Z,[],[],0,0,0,0,0,...,NaN,0.0,NaN,0,False,8.0,NaN,0,False,False


In [26]:
class NeuralNet(torch.nn.Module):
    def __init__(self,input_dim,first_hidden_dim,second_hidden_dim,output_dim,dropout):
        """
        input_dim = dimension of the input : stack between embedded audio and embedded user
        output_dim = output dimension : 1 the overal score 
        first , second : hidden dimension  , architecture is tunable with optuna
        """
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.first_hidden_dim = first_hidden_dim
        self.second_hidden_dim =second_hidden_dim
        self.dropout = torch.nn.Dropout(p=dropout)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        ### this is the architecture of the NN , tunable of course, this is a 1 st shot #####
        self.first_hidden = torch.nn.relu(self.dropout(torch.nn.Linear(self.input_dim,self.first_hidden_dim ) ))
        self.second_hidden = torch.nn.relu( self.dropout(torch.nn.Linear(self.first_hidden_dim , self.second_hidden_dim )) )
        self.output = torch.nn.Linear(self.second_hidden_dim , self.output_dim )
        
    def forward(self,x):
        """
        input_ : input dimension for neural net
        """
        first_hidden = self.first_hidden(x)
        second_hidden = self.second_hidden(first_hidden) 
        output = self.output(second_hidden)
        return torch.nn.relu(output)
    
class Training:
    
    def __init__(self,training_data,test_data,batch_size):
        self.training_data = training_data
        self.test_data = test_data
        self.batch_size = batch_size
    
    def perform_loading(self):
        self.train_dataloader = DataLoader(self.training_data, self.batch_size)
        self.test_dataloader = DataLoader(self.test_data, self.batch_size)
    
    def train(self,model):
        model = NeuralNet().to(device)
        self.loss = torch.nn.MSELoss()
        self.optimizer = torch.optim.SGD(model.parameters(), lr=1e-1   )    
        size = len(self.train_dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(self.train_dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            loss = self.loss(pred, y)

            # Backpropagation
            self.optimizer.zero_grad()
            self.loss.backward()
            self.optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
    def test( self , model):
        size = len(self.test_dataloader.dataset)
        num_batches = len(self.test_dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in self.test_dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += self.loss(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        
        

In [52]:
from torch_geometric.data import Data

class ComputeData(Data):
    
    def __init__(self,df,output_path,column_id,keak_id,target):
        super().__init__()
        self.output_path = output_path
        self.df = df
        self.column_id = column_id 
        self.keak_id = keak_id
        self.target = target
        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return []

    def download(self):
        pass
    
    def process(self):
        
        data_list = []
        item_encoder = LabelEncoder()
        
        df_grouped = self.df.groupby(self.column_id)
        
        for user,group in df_grouped:
            
            user_keak_id = LabelEncoder().fit_transform(group[self.keak_id]) 
            group = group.reset_index(drop=True)
            group['user_keak_id'] = user_keak_id
            node_features = group.loc[group[self.column_id]==user ,['user_keak_id',self.keak_id]].\
                                    sort_values('user_keak_id')[self.keak_id].drop_duplicates().values
            
            node_features = torch.LongTensor(node_features).squeeze(1)
            target_nodes = group.user_keak_id.values[1:]
            source_nodes = group.user_keak_id.values[:-1]
            
            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group[self.target].values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.output_path)
        
        return data,slices

In [53]:
gc.collect()

836

In [54]:
graph = ComputeData(df_audiences , './graph_data.dataset' , 'userId','contentId','liked')

In [55]:
dataset , slices = graph.process()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
dataset = dataset.shuffle()
train_dataset = dataset[:800000]
val_dataset = dataset[800000:900000]
test_dataset = dataset[900000:]

In [ ]:
embed_dim = 128
from torch_geometric.nn import TopKPooling,GATConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F

In [ ]:
class GraphNet(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GATConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = GATConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = GATConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

In [ ]:
def train(model , train_loader):
    
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)
    

In [ ]:
device = torch.device('cuda')
model = GraphNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size)
for epoch in range(num_epochs):
    train(model , train_loader)